### GAMI-Net Example

#### Load and prepared data

In [1]:
import numpy as np
import pandas as pd
from modeva import DataSet
# Loading data into pandas dataframe
df = pd.read_csv("credit_default.csv")
df.columns = ["employment",	"income", "dti", "score", "amount", 
              "tenure",	"emp_length", "delinquencies",
              "savings", "utilization", "default"]

In [2]:
# Create dataset object holder
ds = DataSet()
ds.load_dataframe(data = df) 

✓ Auth code found in local storage.
Authenticating Modeva...
✓ License is active and valid.
✓ Authenticated successfully!


In [3]:
## Preprocess the data
ds.encode_categorical(method="ordinal")
ds.scale_numerical(features=tuple(ds.feature_names_numerical), method="standardize") # standardized numerical features
ds.set_target('default')
ds.preprocess()
## Split data into training and testing sets randomly
ds.set_random_split() 

#### GAMI-Net Training

In [6]:
from modeva.models import MoGAMINetClassifier
model_GAMI = MoGAMINetClassifier(name = "GAMI_Net", feature_names = ds.feature_names,
                                 subnet_size_main_effect=(20,), subnet_size_interaction=(20, 20),
                                 learning_rates=(0.001, 0.001, 0.001),
                                 reg_mono=0.1, mono_increasing_list=(["dti", "amount", 
                                 "delinquencies", "utilization"]), mono_decreasing_list=tuple(["income","score","savings"]))
# train model with input: ds.train_x and target: ds.train_y
model_GAMI.fit(ds.train_x, ds.train_y.ravel())

MoGAMINetClassifier(device='cpu',
                    feature_names=['employment', 'income', 'dti', 'score',
                                   'amount', 'tenure', 'emp_length',
                                   'delinquencies', 'savings', 'utilization'],
                    learning_rates=(0.001, 0.001, 0.001),
                    mono_decreasing_list=('income', 'score', 'savings'),
                    mono_increasing_list=['dti', 'amount', 'delinquencies',
                                          'utilization'],
                    name='GAMI_Net', reg_mono=29074.07974435073)

#### Check Performance and Interpretability

In [7]:
# Create a testsuite that bundles dataset and model
from modeva import TestSuite
ts = TestSuite(ds, model_GAMI) # store bundle of dataset and model in fs
# View model performance metrics
result = ts.diagnose_accuracy_table()
# display the output
result.table

,AUC,ACC,F1,LogLoss,Brier
train,0.745550,0.7835,0.0,0.432903,0.144274
test,0.747317,0.7905,0.0,0.424266,0.141053
GAP,0.001767,0.0070,0.0,-0.008638,-0.003221


In [8]:
results = ts.interpret_fi()
results.plot()

In [9]:
# Check feature effect of most important feature
results = ts.interpret_effects(features = "score")
results.plot(figsize = (6,4))